# K-Nearest Neighbors (KNN) - Complete Guide

## Table of Contents
1. [What is K-Nearest Neighbors?](#what-is-knn)
2. [How KNN Works - The Theory](#how-knn-works)
3. [Visual Demonstrations](#visual-demos)
4. [Distance Metrics and Variants](#distance-metrics)
5. [Practical Examples](#practical-examples)
6. [Performance Analysis](#performance-analysis)
7. [Summary and Key Takeaways](#summary)

---


## 1. What is K-Nearest Neighbors? {#what-is-knn}

**K-Nearest Neighbors (KNN)** is one of the simplest and most intuitive machine learning algorithms. It's a **lazy learning** algorithm that makes predictions based on the similarity to training examples.

### Key Concepts:
- **K**: Number of nearest neighbors to consider
- **Distance Metric**: How to measure similarity between points
- **Lazy Learning**: No explicit training phase - learning happens during prediction
- **Instance-Based**: Stores all training data for predictions

### Why KNN is Popular:
1. **Simple to understand** and implement
2. **No assumptions** about data distribution
3. **Works well** with non-linear data
4. **Versatile** - can be used for both classification and regression
5. **No training time** - learning happens during prediction

### Key Characteristics:
- **Memory intensive** - stores all training data
- **Slow prediction** - must compute distances to all training points
- **Sensitive to irrelevant features** - all features are treated equally
- **Curse of dimensionality** - performance degrades with high dimensions


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification, make_regression
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print("📊 Ready to explore K-Nearest Neighbors!")


## 2. How KNN Works - The Theory {#how-knn-works}

### The Algorithm Steps:

1. **Store all training data** (no learning phase)
2. **For a new prediction**:
   - Calculate distance to all training points
   - Find the K nearest neighbors
   - For classification: majority vote
   - For regression: average of neighbors' values

### Mathematical Foundation:

#### Distance Metrics:
- **Euclidean**: d = √(Σ(xi - yi)²)
- **Manhattan**: d = Σ|xi - yi|
- **Minkowski**: d = (Σ|xi - yi|^p)^(1/p)
- **Cosine**: d = 1 - (x·y)/(||x||·||y||)

#### Classification:
```
Prediction = mode(neighbors_labels)
```

#### Regression:
```
Prediction = mean(neighbors_values)
```

### Key Parameters:
- **K**: Number of neighbors (usually odd for classification)
- **Distance metric**: How to measure similarity
- **Weights**: Uniform or distance-based weighting


## 3. Visual Demonstrations {#visual-demos}

Let's create visual demonstrations to understand KNN concepts better.


In [ ]:
# Create a 2D dataset for KNN visualization
def create_knn_dataset():
    np.random.seed(42)
    
    # Class 1: Blue points
    X1 = np.random.randn(30, 2) + [2, 2]
    y1 = np.ones(30)
    
    # Class 2: Red points  
    X2 = np.random.randn(30, 2) + [-2, -2]
    y2 = -np.ones(30)
    
    # Class 3: Green points
    X3 = np.random.randn(30, 2) + [2, -2]
    y3 = np.zeros(30)
    
    X = np.vstack([X1, X2, X3])
    y = np.hstack([y1, y2, y3])
    
    return X, y

X_knn, y_knn = create_knn_dataset()
print(f"Dataset shape: {X_knn.shape}")
print(f"Classes: {np.unique(y_knn)}")

# Visualize the dataset
plt.figure(figsize=(10, 8))
colors = ['red' if label == -1 else 'blue' if label == 1 else 'green' for label in y_knn]
plt.scatter(X_knn[:, 0], X_knn[:, 1], c=colors, alpha=0.7, s=100, edgecolors='black')

plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title('Sample Dataset for KNN Demonstration', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Add legend
plt.scatter([], [], c='red', label='Class -1', s=100)
plt.scatter([], [], c='blue', label='Class +1', s=100)
plt.scatter([], [], c='green', label='Class 0', s=100)
plt.legend(fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
# Visualize KNN decision boundaries for different K values
def plot_knn_decision_boundary(X, y, k_values=[1, 3, 5, 15]):
    """Plot KNN decision boundaries for different K values"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.ravel()
    
    # Create a mesh grid
    h = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    for i, k in enumerate(k_values):
        # Train KNN
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X, y)
        
        # Make predictions on the mesh grid
        Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        
        # Plot decision boundary
        axes[i].contourf(xx, yy, Z, alpha=0.3, cmap='viridis')
        
        # Plot data points
        colors = ['red' if label == -1 else 'blue' if label == 1 else 'green' for label in y]
        axes[i].scatter(X[:, 0], X[:, 1], c=colors, alpha=0.8, s=60, edgecolors='black')
        
        # Calculate accuracy
        accuracy = knn.score(X, y)
        
        axes[i].set_title(f'K = {k}\nAccuracy: {accuracy:.3f}', fontweight='bold')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("🔍 Effect of K parameter on KNN:")
print("• K = 1: Very sensitive to noise, overfitting")
print("• K = 3: Balanced, good for most cases")
print("• K = 5: Smoother boundaries, less noise sensitive")
print("• K = 15: Very smooth, may underfit")

plot_knn_decision_boundary(X_knn, y_knn)


In [ ]:
# Demonstrate how KNN finds nearest neighbors
def demonstrate_nearest_neighbors(X, y, test_point, k=5):
    """Show how KNN finds and uses nearest neighbors"""
    # Train KNN
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X, y)
    
    # Find nearest neighbors
    distances, indices = knn.kneighbors([test_point])
    
    # Create visualization
    plt.figure(figsize=(12, 5))
    
    # Plot 1: All data points
    plt.subplot(1, 2, 1)
    colors = ['red' if label == -1 else 'blue' if label == 1 else 'green' for label in y]
    plt.scatter(X[:, 0], X[:, 1], c=colors, alpha=0.6, s=80, edgecolors='black')
    
    # Highlight test point
    plt.scatter(test_point[0], test_point[1], c='black', s=200, marker='x', linewidth=3, label='Test Point')
    
    # Highlight nearest neighbors
    neighbors = X[indices[0]]
    neighbor_labels = y[indices[0]]
    neighbor_colors = ['red' if label == -1 else 'blue' if label == 1 else 'green' for label in neighbor_labels]
    plt.scatter(neighbors[:, 0], neighbors[:, 1], c=neighbor_colors, s=150, edgecolors='yellow', linewidth=2, label=f'K={k} Neighbors')
    
    # Draw circles around neighbors
    for i, (x, y) in enumerate(neighbors):
        circle = plt.Circle((x, y), distances[0][i], fill=False, color='yellow', alpha=0.5, linewidth=2)
        plt.gca().add_patch(circle)
    
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title('KNN: Finding Nearest Neighbors')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Distance distribution
    plt.subplot(1, 2, 2)
    plt.bar(range(1, k+1), distances[0], color='skyblue', alpha=0.7)
    plt.xlabel('Neighbor Rank')
    plt.ylabel('Distance')
    plt.title('Distance to K Nearest Neighbors')
    plt.grid(True, alpha=0.3)
    
    # Show prediction
    prediction = knn.predict([test_point])[0]
    neighbor_votes = neighbor_labels
    vote_counts = {label: np.sum(neighbor_votes == label) for label in np.unique(neighbor_votes)}
    
    print(f"🎯 Test Point: {test_point}")
    print(f"📊 Nearest Neighbors: {k}")
    print(f"📏 Distances: {distances[0]}")
    print(f"🏷️  Neighbor Labels: {neighbor_votes}")
    print(f"🗳️  Vote Counts: {vote_counts}")
    print(f"🎉 Prediction: {prediction}")
    
    plt.tight_layout()
    plt.show()

# Test with a specific point
test_point = [0, 0]
demonstrate_nearest_neighbors(X_knn, y_knn, test_point, k=5)


## 4. Distance Metrics and Variants {#distance-metrics}

KNN's performance heavily depends on the distance metric used. Let's explore different options:


In [ ]:
# Compare different distance metrics
def compare_distance_metrics(X, y):
    """Compare different distance metrics for KNN"""
    metrics = ['euclidean', 'manhattan', 'chebyshev', 'minkowski']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    results = {}
    
    print("📏 Distance Metrics Comparison:")
    print("=" * 50)
    print(f"{'Metric':<12} {'Accuracy':<10} {'Training Time':<15}")
    print("-" * 50)
    
    for metric in metrics:
        # Train KNN with different metrics
        knn = KNeighborsClassifier(n_neighbors=5, metric=metric)
        
        # Measure training time
        import time
        start_time = time.time()
        knn.fit(X_train, y_train)
        training_time = time.time() - start_time
        
        # Calculate accuracy
        accuracy = knn.score(X_test, y_test)
        
        results[metric] = {
            'accuracy': accuracy,
            'time': training_time,
            'model': knn
        }
        
        print(f"{metric:<12} {accuracy:<10.3f} {training_time:<15.4f}")
    
    return results

# Run comparison
distance_results = compare_distance_metrics(X_knn, y_knn)


In [ ]:
# Visualize decision boundaries for different distance metrics
def plot_distance_metrics_decision_boundaries(X, y, results):
    """Plot decision boundaries for different distance metrics"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.ravel()
    
    # Create a mesh grid
    h = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    for i, (metric, result) in enumerate(results.items()):
        model = result['model']
        accuracy = result['accuracy']
        
        # Make predictions on the mesh grid
        Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        
        # Plot decision boundary
        axes[i].contourf(xx, yy, Z, alpha=0.3, cmap='viridis')
        
        # Plot data points
        colors = ['red' if label == -1 else 'blue' if label == 1 else 'green' for label in y]
        axes[i].scatter(X[:, 0], X[:, 1], c=colors, alpha=0.8, s=60, edgecolors='black')
        
        axes[i].set_title(f'{metric.capitalize()}\nAccuracy: {accuracy:.3f}', fontweight='bold')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_distance_metrics_decision_boundaries(X_knn, y_knn, distance_results)


### Distance Metrics Explained:

#### 1. **Euclidean Distance** (L2)
- **Formula**: √(Σ(xi - yi)²)
- **Best for**: Continuous features, when all features are equally important
- **Sensitive to**: Scale differences between features

#### 2. **Manhattan Distance** (L1)
- **Formula**: Σ|xi - yi|
- **Best for**: Categorical features, when features are independent
- **Less sensitive to**: Outliers compared to Euclidean

#### 3. **Chebyshev Distance** (L∞)
- **Formula**: max|xi - yi|
- **Best for**: When only the maximum difference matters
- **Use case**: Chess moves, image processing

#### 4. **Minkowski Distance** (Lp)
- **Formula**: (Σ|xi - yi|^p)^(1/p)
- **Generalization**: p=1 (Manhattan), p=2 (Euclidean), p=∞ (Chebyshev)
- **Flexible**: Can be tuned for specific applications


## 5. Practical Examples {#practical-examples}

Let's work with real datasets to see KNN in action!


In [ ]:
# Load and work with the Iris dataset
iris = datasets.load_iris()
X_iris = iris.data
y_iris = iris.target

print("🌸 Iris Dataset Information:")
print(f"Features: {iris.feature_names}")
print(f"Classes: {iris.target_names}")
print(f"Samples: {X_iris.shape[0]}")
print(f"Features: {X_iris.shape[1]}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, test_size=0.3, random_state=42)

# Scale the features (important for KNN!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n📊 Data Split:")
print(f"Training samples: {X_train_scaled.shape[0]}")
print(f"Test samples: {X_test_scaled.shape[0]}")
print(f"Features: {X_train_scaled.shape[1]}")

# Train KNN on Iris dataset
knn_iris = KNeighborsClassifier(n_neighbors=5)
knn_iris.fit(X_train_scaled, y_train)

# Make predictions
y_pred = knn_iris.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print(f"\n🎯 KNN Performance on Iris Dataset:")
print(f"Accuracy: {accuracy:.3f}")
print(f"K value: 5")
print(f"Distance metric: Euclidean (default)")

# Classification report
print(f"\n📋 Detailed Classification Report:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))


In [ ]:
# KNN for Regression - Create synthetic regression data
def demonstrate_knn_regression():
    """Demonstrate KNN for regression tasks"""
    # Create regression dataset
    X_reg, y_reg = make_regression(n_samples=100, n_features=1, noise=10, random_state=42)
    
    # Sort for better visualization
    sort_idx = np.argsort(X_reg.ravel())
    X_reg = X_reg[sort_idx]
    y_reg = y_reg[sort_idx]
    
    # Split data
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
        X_reg, y_reg, test_size=0.3, random_state=42
    )
    
    # Train KNN regressor
    knn_reg = KNeighborsRegressor(n_neighbors=5)
    knn_reg.fit(X_train_reg, y_train_reg)
    
    # Make predictions
    y_pred_reg = knn_reg.predict(X_test_reg)
    
    # Calculate R² score
    from sklearn.metrics import r2_score
    r2 = r2_score(y_test_reg, y_pred_reg)
    
    # Visualize
    plt.figure(figsize=(12, 5))
    
    # Plot 1: Training data and predictions
    plt.subplot(1, 2, 1)
    plt.scatter(X_train_reg, y_train_reg, alpha=0.6, label='Training Data', color='blue')
    plt.scatter(X_test_reg, y_test_reg, alpha=0.8, label='Test Data', color='red', s=100)
    plt.scatter(X_test_reg, y_pred_reg, alpha=0.8, label='KNN Predictions', color='green', s=100)
    plt.xlabel('Feature')
    plt.ylabel('Target')
    plt.title('KNN Regression')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Prediction vs Actual
    plt.subplot(1, 2, 2)
    plt.scatter(y_test_reg, y_pred_reg, alpha=0.7)
    plt.plot([y_test_reg.min(), y_test_reg.max()], [y_test_reg.min(), y_test_reg.max()], 'r--', lw=2)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Prediction vs Actual (R² = {r2:.3f})')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"📈 KNN Regression Results:")
    print(f"R² Score: {r2:.3f}")
    print(f"K value: 5")
    print(f"Training samples: {len(X_train_reg)}")
    print(f"Test samples: {len(X_test_reg)}")

demonstrate_knn_regression()


## 6. Performance Analysis {#performance-analysis}

Let's analyze KNN's performance characteristics and compare different approaches.


In [ ]:
# Analyze the effect of K on performance
def analyze_k_performance(X, y):
    """Analyze how different K values affect performance"""
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Test different K values
    k_values = range(1, 21)
    train_scores = []
    test_scores = []
    
    for k in k_values:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train_scaled, y_train)
        
        train_score = knn.score(X_train_scaled, y_train)
        test_score = knn.score(X_test_scaled, y_test)
        
        train_scores.append(train_score)
        test_scores.append(test_score)
    
    # Plot results
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(k_values, train_scores, 'o-', label='Training Score', linewidth=2)
    plt.plot(k_values, test_scores, 's-', label='Test Score', linewidth=2)
    plt.xlabel('K Value')
    plt.ylabel('Accuracy')
    plt.title('KNN Performance vs K Value')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Find optimal K
    optimal_k = k_values[np.argmax(test_scores)]
    optimal_score = max(test_scores)
    
    plt.axvline(x=optimal_k, color='red', linestyle='--', alpha=0.7, label=f'Optimal K = {optimal_k}')
    plt.legend()
    
    # Plot bias-variance tradeoff
    plt.subplot(1, 2, 2)
    bias = [1 - score for score in train_scores]
    variance = [test - train for test, train in zip(test_scores, train_scores)]
    
    plt.plot(k_values, bias, 'o-', label='Bias (1 - Training Score)', linewidth=2)
    plt.plot(k_values, variance, 's-', label='Variance (Test - Training)', linewidth=2)
    plt.xlabel('K Value')
    plt.ylabel('Error')
    plt.title('Bias-Variance Tradeoff')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"🎯 Performance Analysis Results:")
    print(f"Optimal K: {optimal_k}")
    print(f"Best Test Score: {optimal_score:.3f}")
    print(f"Training Score at Optimal K: {train_scores[optimal_k-1]:.3f}")
    
    return k_values, train_scores, test_scores, optimal_k

# Run analysis
k_values, train_scores, test_scores, optimal_k = analyze_k_performance(X_iris, y_iris)


In [ ]:
# Cross-validation analysis
def cross_validation_analysis(X, y):
    """Perform cross-validation analysis for KNN"""
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Test different K values with cross-validation
    k_values = range(1, 21)
    cv_scores = []
    
    for k in k_values:
        knn = KNeighborsClassifier(n_neighbors=k)
        scores = cross_val_score(knn, X_scaled, y, cv=5, scoring='accuracy')
        cv_scores.append(scores.mean())
    
    # Plot cross-validation results
    plt.figure(figsize=(10, 6))
    plt.plot(k_values, cv_scores, 'o-', linewidth=2, markersize=6)
    plt.xlabel('K Value')
    plt.ylabel('Cross-Validation Accuracy')
    plt.title('KNN Cross-Validation Performance')
    plt.grid(True, alpha=0.3)
    
    # Find optimal K
    optimal_k_cv = k_values[np.argmax(cv_scores)]
    optimal_score_cv = max(cv_scores)
    
    plt.axvline(x=optimal_k_cv, color='red', linestyle='--', alpha=0.7, 
                label=f'Optimal K = {optimal_k_cv}')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"🔄 Cross-Validation Results:")
    print(f"Optimal K: {optimal_k_cv}")
    print(f"Best CV Score: {optimal_score_cv:.3f}")
    
    return k_values, cv_scores, optimal_k_cv

# Run cross-validation analysis
k_values_cv, cv_scores, optimal_k_cv = cross_validation_analysis(X_iris, y_iris)


## 7. Summary and Key Takeaways {#summary}

### 🎯 What We Learned:

1. **KNN Fundamentals**:
   - Simple, intuitive algorithm based on similarity
   - Lazy learning - no explicit training phase
   - Works for both classification and regression

2. **Key Parameters**:
   - **K**: Number of neighbors (affects bias-variance tradeoff)
   - **Distance metric**: How to measure similarity
   - **Feature scaling**: Critical for good performance

3. **Performance Characteristics**:
   - **Memory intensive**: Stores all training data
   - **Slow prediction**: Must compute distances to all points
   - **Sensitive to irrelevant features**: All features treated equally
   - **Curse of dimensionality**: Performance degrades with high dimensions

### 🚀 When to Use KNN:

✅ **Good for**:
- Small to medium datasets
- Non-linear decision boundaries
- Multi-class problems
- When you need interpretable results
- Prototype-based learning

❌ **Not ideal for**:
- Large datasets (slow prediction)
- High-dimensional data (curse of dimensionality)
- When you need fast prediction
- When memory is limited
- Noisy data with many irrelevant features

### 🔧 Key Parameters to Tune:

1. **K**: Start with √n (where n = number of samples)
2. **Distance metric**: Euclidean for continuous, Manhattan for categorical
3. **Feature scaling**: Always scale features before using KNN
4. **Weights**: Uniform or distance-based weighting

### 📚 Best Practices:

1. **Always scale features** - KNN is sensitive to feature scales
2. **Use cross-validation** to find optimal K
3. **Consider feature selection** to reduce dimensionality
4. **Use odd K values** for classification to avoid ties
5. **Consider computational cost** for large datasets

### 🆚 KNN vs Other Algorithms:

| Aspect | KNN | SVM | Random Forest |
|--------|-----|-----|---------------|
| **Training time** | None | Fast | Medium |
| **Prediction time** | Slow | Fast | Fast |
| **Memory usage** | High | Low | Medium |
| **Interpretability** | High | Medium | Medium |
| **Non-linear data** | Excellent | Excellent | Excellent |
| **High dimensions** | Poor | Excellent | Good |

### 📈 Next Steps:

1. Try KNN on your own datasets
2. Experiment with different distance metrics
3. Compare with other algorithms
4. Explore advanced techniques like KD-trees
5. Learn about weighted KNN and distance weighting

---

**Congratulations! 🎉 You now understand K-Nearest Neighbors!**

KNN is a powerful, intuitive algorithm that's perfect for understanding the fundamentals of machine learning. While it has limitations with large datasets and high dimensions, it's excellent for smaller problems and provides valuable insights into similarity-based learning.
